# Feature Engineering

The next step is to create features from the raw text so we can train the machine learning models. The steps followed are:

1. **Text Cleaning and Preparation**: cleaning of special characters, downcasing, punctuation signs. possessive pronouns and stop words removal and lemmatization. 
2. **Label coding**: creation of a dictionary to map each category to a code.
3. **Train-test split**: to test the models on unseen data.
4. **Text representation**: use of TF-IDF scores to represent text.

In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

First of all we'll load the dataset:

In [4]:
path_df = "Pickles/Train_all_types.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [5]:
df.head()

,Complexity_level,text,id,node_length
0,L-Q,is moss a plant or a fungus\n[ - is that it is...,1,149
1,L-Q,It was fun to do with classmates and easy it d...,1,164
2,L-Q,Maybe there is something attracting the moss s...,1,62
3,L-Q,[are ant hills living or noliving-- ]\n,1,38
4,L-Q,Did you know that there is a Queen ant?,1,39


And visualize one sample news content:

In [7]:
df.loc[1]['text']

"It was fun to do with classmates and easy it didn't really make a big mess for us my questions are can we go longer and can it get harder cuz what we did was easy ."

## 1. Text cleaning and preparation

### 1.1. Special character cleaning

We can see the following special characters:

* ``\r``
* ``\n``
* ``\`` before possessive pronouns (`government's = government\'s`)
* ``\`` before possessive pronouns 2 (`Yukos'` = `Yukos\'`)
* ``"`` when quoting text

In [8]:
# \r and \n
df['Content_Parsed_1'] = df['text'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

Regarding 3rd and 4th bullet, although it seems there is a special character, it won't affect us since it is not a *real* character:

In [9]:
text = "Mr Greenspan\'s"
text

"Mr Greenspan's"

In [10]:
# " when quoting text
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

### 1.2. Upcase/downcase

We'll downcase the texts because we want, for example, `Football` and `football` to be the same word.

In [11]:
# Lowercasing the text
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

### 1.3. Punctuation signs

Punctuation signs won't have any predicting power, so we'll just get rid of them.

In [12]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

By doing this we are messing up with some numbers, but it's no problem since we aren't expecting any predicting power from them.

### 1.4. Possessive pronouns

We'll also remove possessive pronoun terminations:

In [13]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

### 1.5. Stemming and Lemmatization

Since stemming can produce output words that don't exist, we'll only use a lemmatization process at this moment. Lemmatization takes into consideration the morphological analysis of the words and returns words that do exist, so it will be more useful for us.

In [14]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jzhong\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jzhong\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In order to lemmatize, we have to iterate through every word:

In [16]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [17]:
df['Content_Parsed_5'] = lemmatized_text_list

Although lemmatization doesn't work perfectly in all cases (as can be seen in the example below), it can be useful.

### 1.6. Stop words

In [18]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jzhong\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [19]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

In [20]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

To remove the stop words, we'll handle a regular expression only detecting whole words, as seen in the following example:

In [21]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

We can now loop through all the stop words:

In [22]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

We have some dobule/triple spaces between words because of the replacements. However, it's not a problem because we'll tokenize by the spaces later.

As an example, we'll show an original news article and its modifications throughout the process:

In [23]:
df.loc[5]['text']

'In the court yard the apple tree is tilted did it gorw like that?'

1. Special character cleaning

In [24]:
df.loc[5]['Content_Parsed_1']

'In the court yard the apple tree is tilted did it gorw like that?'

2. Upcase/downcase

In [25]:
df.loc[5]['Content_Parsed_2']

'in the court yard the apple tree is tilted did it gorw like that?'

3. Punctuation signs

In [26]:
df.loc[5]['Content_Parsed_3']

'in the court yard the apple tree is tilted did it gorw like that'

4. Possessive pronouns

In [27]:
df.loc[5]['Content_Parsed_4']

'in the court yard the apple tree is tilted did it gorw like that'

5. Stemming and Lemmatization

In [28]:
df.loc[5]['Content_Parsed_5']

'in the court yard the apple tree be tilt do it gorw like that'

6. Stop words

In [36]:
df.loc[5]['Content_Parsed_6']

KeyError: 'Content_Parsed_6'

Finally, we can delete the intermediate columns:

In [37]:
df.head(1)

,Complexity_level,text,Content_Parsed
0,L-Q,is moss a plant or a fungus\n[ - is that it is...,moss plant fungus [ - fungus- ] un...


In [38]:
list_columns = ["Complexity_level", "text", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

KeyError: "['Content_Parsed_6'] not in index"

In [40]:
df.head()

,Complexity_level,text,Content_Parsed
0,L-Q,is moss a plant or a fungus\n[ - is that it is...,moss plant fungus [ - fungus- ] un...
1,L-Q,It was fun to do with classmates and easy it d...,fun classmates easy ' really make big ...
2,L-Q,Maybe there is something attracting the moss s...,maybe something attract moss go north
3,L-Q,[are ant hills living or noliving-- ]\n,[ ant hill live noliving-- ]
4,L-Q,Did you know that there is a Queen ant?,know queen ant


**IMPORTANT:**

We need to remember that our model will gather the latest news articles from different newspapers every time we want. For that reason, we not only need to take into account the peculiarities of the training set articles, but also possible ones that are present in the gathered news articles.

For this reason, possible peculiarities have been studied in the *05. News Scraping* folder.

## 2. Label coding

We'll create a dictionary with the label codification:

In [41]:
complexity_level = {
    'L-I': 0,
    'L-IS': 1,
    'L-Q': 2,
    'L-R': 3
}

In [42]:
# Category mapping
df['Complexity'] = df['Complexity_level']
df = df.replace({'Complexity':complexity_level})

In [43]:
df.head()

,Complexity_level,text,Content_Parsed,Complexity
0,L-Q,is moss a plant or a fungus\n[ - is that it is...,moss plant fungus [ - fungus- ] un...,2
1,L-Q,It was fun to do with classmates and easy it d...,fun classmates easy ' really make big ...,2
2,L-Q,Maybe there is something attracting the moss s...,maybe something attract moss go north,2
3,L-Q,[are ant hills living or noliving-- ]\n,[ ant hill live noliving-- ],2
4,L-Q,Did you know that there is a Queen ant?,know queen ant,2


## 3. Train - test split

We'll set apart a test set to prove the quality of our models. We'll do Cross Validation in the train set in order to tune the hyperparameters and then test performance on the unseen data of the test set.

In [44]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Complexity'], 
                                                    test_size=0.15, 
                                                    random_state=8)

Since we don't have much observations (only 2.225), we'll choose a test set size of 15% of the full dataset.

## 4. Text representation

We have various options:

* Count Vectors as features
* TF-IDF Vectors as features
* Word Embeddings as features
* Text / NLP based features
* Topic Models as features

We'll use **TF-IDF Vectors** as features.

We have to define the different parameters:

* `ngram_range`: We want to consider both unigrams and bigrams.
* `max_df`: When building the vocabulary ignore terms that have a document
    frequency strictly higher than the given threshold
* `min_df`: When building the vocabulary ignore terms that have a document
    frequency strictly lower than the given threshold.
* `max_features`: If not None, build a vocabulary that only consider the top
    max_features ordered by term frequency across the corpus.

See `TfidfVectorizer?` for further detail.

It needs to be mentioned that we are implicitly scaling our data when representing it as TF-IDF features with the argument `norm`.

In [45]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

We have chosen these values as a first approximation. Since the models that we develop later have a very good predictive power, we'll stick to these values. But it has to be mentioned that different combinations could be tried in order to improve even more the accuracy of the models.

In [47]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)
complexity_level.items()

(1110, 300)
(196, 300)


dict_items([('L-I', 0), ('L-IS', 1), ('L-Q', 2), ('L-R', 3)])

Please note that we have fitted and then transformed the training set, but we have **only transformed** the **test set**.

We can use the Chi squared test in order to see what unigrams and bigrams are most correlated with each category:

In [48]:
from sklearn.feature_selection import chi2
import numpy as np

for Complexity_level, Complexity in sorted(complexity_level.items()):
    features_chi2 = chi2(features_train, labels_train == Complexity)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' complexity:".format(Complexity_level))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")


# 'L-I' complexity:
  . Most correlated unigrams:
. understand
. docsgooglecom
. wonder
. link
. https
  . Most correlated bigrams:
. need understand
. link https

# 'L-IS' complexity:
  . Most correlated unigrams:
. put
. together
. know
. dont
. dark
  . Most correlated bigrams:
. link https
. need understand

# 'L-Q' complexity:
  . Most correlated unigrams:
. courtyard
. light
. need
. understand
. wonder
  . Most correlated bigrams:
. make water
. need understand

# 'L-R' complexity:
  . Most correlated unigrams:
. wwwbrainpopcom
. click
. docsgooglecom
. link
. https
  . Most correlated bigrams:
. https docsgooglecom
. link https



As we can see, the unigrams correspond well to their category. However, bigrams do not. If we get the bigrams in our features:

In [49]:
bigrams

['make water',
 'look like',
 'moss grow',
 'need water',
 'make food',
 'theory cannot',
 'ants live',
 'cannot explain',
 'apple tree',
 'concave mirror',
 'red rise',
 'carbon dioxide',
 'green blue',
 'get water',
 'blue green',
 'salt water',
 'red green',
 'lunar eclipse',
 'solar eclipse',
 'opaque object',
 'make rainbow',
 'build ant',
 'white light',
 'light hit',
 'color light',
 'new information',
 'ant hill',
 'need understand',
 'kf6ritalbanyedu attachments',
 'attachments 5b87586745b8a02554df72de',
 'https kf6ritalbanyedu',
 'image https',
 'docsgooglecom presentation',
 'https docsgooglecom',
 'link https']

We can see there are only six. This means the unigrams have more correlation with the category than the bigrams, and since we're restricting the number of features to the most representative 300, only a few bigrams are being considered.

Let's save the files we'll need in the next steps:

In [50]:
# X_train
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)